In [1]:
import sys
import logging
import os
import time
import boto3
import json

In [2]:
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 

In [3]:
kb_name = 'demo-knowledge-base-csv'
kbs = bedrock_agent_client.list_knowledge_bases(maxResults=100)
kb_id = next((kb['knowledgeBaseId'] for kb in kbs['knowledgeBaseSummaries'] if kb['name'] == kb_name), None)

In [4]:
query = "What are key drink categories?"
fm = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{fm}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                }
            }
        }
    }
)
print(f"Response status: {response['ResponseMetadata']['HTTPStatusCode']}")

Response status: 200


In [5]:
print(f"""
Here is the query:
{query}

Here is the NL output:
{response['output']['text']}

Here is the SQL to retrieve data from CSV files:
{response['citations'][0]['retrievedReferences'][0]['location']['sqlLocation']['query']}
""")


Here is the query:
What are key drink categories?

Here is the NL output:
The key drink categories based on the query output are:
- Espresso (with both Decaf and Regular caffeination)
- Coffee (with both Decaf and Regular caffeination)
- Tea (Regular caffeination)
- Herbal Tea (Decaf caffeination)

Here is the SQL to retrieve data from CSV files:
SELECT DISTINCT "category", "caffeination" FROM awsdatacatalog."demo-kb".demo_csv_639649899411;



In [6]:
print(f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}")

arn:aws:bedrock:us-east-1:639649899411:knowledge-base/FJG5JJM6J1


In [7]:
query = "What are key drink categories?"

response = bedrock_agent_runtime_client.generate_query(
    queryGenerationInput={
        'text': query,
        'type': 'TEXT'
    },
    transformationConfiguration={
        'mode': 'TEXT_TO_SQL',
        'textToSqlConfiguration': {
            'knowledgeBaseConfiguration': {
                'knowledgeBaseArn': f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
            },
            'type': 'KNOWLEDGE_BASE'
        }
    }
)

print(f"Response status: {response['ResponseMetadata']['HTTPStatusCode']}")

Response status: 200


In [8]:
print(f"""
Here is the query:
{query}

Here is the SQL to retrieve data from CSV files:
{response['queries'][0]['sql']}
""")


Here is the query:
What are key drink categories?

Here is the SQL to retrieve data from CSV files:
SELECT DISTINCT "category", "caffeination" FROM awsdatacatalog."demo-kb".demo_csv_639649899411;

